# Advanced `pandas`

The following notebook is dedicated to more advanved opeartions in Pandas:

- `split-apply-combine` pipeline,
- operations on string columns (string operations, replacement),
- joins on Pandas dataframes.

In [ ]:
%pylab inline
plt.style.use("bmh")

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
titanic_train = pd.read_csv("train.csv", index_col="PassengerId")
titanic_test = pd.read_csv("test.csv", index_col="PassengerId")
titanic = pd.concat([titanic_train, titanic_test], sort=False)

In [ ]:
titanic.head()

In [ ]:
titanic.groupby("Ticket").size().value_counts()

In [ ]:
titanic.groupby("Ticket")["Fare"].mean() / titanic.groupby("Ticket").size()

In [ ]:
titanic.merge((titanic.groupby("Ticket")["Fare"].mean() / titanic.groupby("Ticket").size()).rename("fare_per_pass"),
              left_on="Ticket", right_index=True, how="left").groupby("Pclass")["fare_per_pass"].mean()

# Split-apply-combine (`GROUP BY` in Pandas)

Depending on how the result of `apply` part of the pipeline is structured, Pandas will `combine` differently. Many common operations have shortcuts, making them extremely concise. We start with the most simple case: `apply` results in a single scalar per group.

Entry point to Pandas grouping:

In [ ]:
class_groups = titanic.groupby("Pclass")

In [ ]:
class_groups

Pandas is smart enough to provide some common operations automatically:

In [ ]:
class_groups.mean()

We can group (only a single column) by a synthetic key:

In [ ]:
age_groups = titanic.Parch.groupby((5 + 10*(titanic.Age//10)))

In [ ]:
5 + 10*(titanic.Age//10)

In [ ]:
age_groups

In [ ]:
age_groups.mean().to_frame() # Note index name

NumPy arrays can also be used as grouping keys:

In [ ]:
age_groups_npy = titanic.Parch.groupby((5 + 10*(titanic.Age//10)).values)

In [ ]:
age_groups_npy.mean().to_frame()

We can group by a set of keys:

In [ ]:
age_groups_multi = titanic.Parch.groupby([(5 + 10*(titanic.Age//10)), titanic.Pclass])

In [ ]:
age_groups_multi = titanic.Parch.groupby([titanic.Pclass, (5 + 10*(titanic.Age//10))])

In [ ]:
age_groups_multi

In [ ]:
age_groups_multi.mean()

We can restructure the result:

In [ ]:
age_groups_multi.mean().unstack()

We can mix column names with a real iterables:

In [ ]:
age_groups_mixed = titanic.groupby([(5 + 10*(titanic.Age//10)), "Pclass", "Embarked"])

In [ ]:
age_groups_mixed

In [ ]:
age_groups_mixed.Parch.mean()

In [ ]:
age_groups_mixed.Parch.mean().unstack(level=(0,2))

## `apply` outputs series

In [ ]:
class_groups = titanic.groupby("Pclass") # Nothing is calculated yet

In [ ]:
class_groups.mean()

Let's simulate series output of `apply` stage:

In [ ]:
titanic.Fare.describe()

In [ ]:
class_groups.Fare.describe()

In [ ]:
class_groups.apply(lambda x: x.Fare.describe()) # Note column index name

In [ ]:
titanic.groupby("Sex").Parch.mean()

In [ ]:
class_groups.apply(lambda x: x.groupby("Sex").Parch.mean()) # Note column index name

In [ ]:
class_groups.apply(lambda x: x[x.Parch==0].groupby("Sex").size())

The same can be achieved differently, of course:

In [ ]:
titanic[titanic.Parch==0].groupby(["Pclass", "Sex"]).size().unstack()

What if `apply` result has multi-index on it's own?

In [ ]:
titanic[titanic.Parch!=0].groupby(["Sex", "Embarked"]).size()

In [ ]:
class_groups.apply(lambda x: x[x.Parch!=0].groupby(["Sex", "Embarked"]).size())

In [ ]:
result_s = (class_groups
            .apply(lambda x: x[x.Parch!=0].groupby(["Sex", "Embarked"]).size()))

In [ ]:
result = (class_groups
          .apply(lambda x: x[x.Parch!=0].groupby(["Sex", "Embarked"]).size())
          .unstack([1,2]))

In [ ]:
result

### Intermezzo: indexing a multi-indexed dataframe

In [ ]:
result.loc[:, ("female", "C")]

In [ ]:
result.loc[:, [("female", "C"), ("female", "S")]]

Positional index is slightly different, as it known nothing about multi-index *per se*:

In [ ]:
result.iloc[:, [0, 1]]

## DataFrame output

In [ ]:
titanic[["SibSp", "Parch"]].head()

In [ ]:
(titanic[titanic.Parch!=0]
 .groupby(["Sex", "Embarked"])[["SibSp", "Parch"]]
 .mean())

In [ ]:
class_groups

In [ ]:
(class_groups
 .apply(lambda x: x[x.Parch!=0].groupby(["Sex", "Embarked"])[["SibSp", "Parch"]]
        .mean()))

## Mixing group keys

In [ ]:
titanic.head()

In [ ]:
titanic_idx = titanic.reset_index().set_index((5 + 10*(titanic.Age//10)))

In [ ]:
titanic_idx.head()

`pd.Grouper` is an entry point to complex mixed groupings:

In [ ]:
pd.Grouper?

In [ ]:
titanic_idx.groupby([pd.Grouper(level="Age"), "Pclass"]).Parch.mean()

In [ ]:
titanic_idx.index

In [ ]:
titanic_idx.groupby([titanic_idx.index, "Pclass"]).Parch.mean()

### Intermezzo: on `size` vs. `count`

`size` is a method to get, you name it, **size** of something, in this case, of a group:

In [ ]:
titanic.groupby('Pclass').size()  ## how many elements are in each group

But you **count** only something specific:

In [ ]:
titanic.groupby('Pclass').count()

As you can see, `count` only counts non-missing values, i.e. something, that is present in the dataframe. Hence, a bit more elaborated way of getting (almost) the same is:

In [ ]:
titanic.isnull()

In [ ]:
titanic.groupby('Pclass').apply(lambda group: group.notnull().sum())

# How `S-A-C` is important in exploratory data analysis

In [ ]:
titanic.Pclass.value_counts()

In [ ]:
titanic["AgeGroup"] = 5 + 10*(titanic.Age//10)
titanic[["Age", "AgeGroup"]]

Let's calculate something non-trivial. For example, percentage of each age group and sex combination, per class:

In [ ]:
titanic_dna = titanic[titanic.Age.notnull()]  # it's a matter of discussion, if we need this: think on it a bit
group_counts = titanic_dna.groupby(['Pclass', 'AgeGroup', 'Sex']).size()/titanic_dna.groupby('Pclass').size()
group_counts.head()

In [ ]:
group_counts.unstack(level=1)

A simple way to validate the calculation:

In [ ]:
group_counts.groupby(level=0).sum()

Main advantage of Pandas for EDA comes from very flexible inter-operability of analytics and plotting:

In [ ]:
group_counts = group_counts.unstack()

In [ ]:
group_counts

In [ ]:
plt.figure(figsize=(15, 5))

for pclass in [1, 2, 3]:
    plt.subplot(1, 3, pclass)
    group_counts.loc[pclass].plot(ax=plt.gca())
    plt.ylim(0, 0.35)
    plt.title("Age distribution for Class %i" % pclass, fontsize=12)
plt.tight_layout()

In [ ]:
survival_groups = titanic.groupby(['Pclass', 'AgeGroup', 'Sex']).Survived.mean()
survival_groups

In [ ]:
survival_groups = survival_groups.unstack()

In [ ]:
plt.figure(figsize=(15, 5))

for pclass in [1, 2, 3]:
    plt.subplot(1, 3, pclass)
    survival_groups.loc[pclass].plot(ax=plt.gca())
    plt.ylim(0, 1)
    plt.title("Survived in class %i" % pclass, fontsize=12)
plt.tight_layout()

In [ ]:
siblings_groups = titanic.groupby(['Pclass', 'AgeGroup', 'Sex']).SibSp.mean()
siblings_groups = siblings_groups.unstack()

In [ ]:
siblings_groups

In [ ]:
plt.figure(figsize=(15, 5))

for pclass in [1, 2, 3]:
    plt.subplot(1, 3, pclass)
    siblings_groups.loc[pclass].plot(ax=plt.gca())
    plt.ylim(0, 5)
    plt.title("Siblings in class %i" % pclass, fontsize=12)
plt.tight_layout()

In [ ]:
embark_counts = titanic.groupby(['Pclass', 'AgeGroup', 'Sex', 'Embarked']).size()/titanic.groupby('Pclass').size()

embark_counts

In [ ]:
embark_counts = embark_counts.unstack([-1, -2])
embark_counts

### Intermezzo: Seaborn in EDA

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize=(6,6))

# We will plot jittered version of the data, hence we remove fliers
sns.boxplot(x="Pclass", y="Age", data=titanic,
            fliersize=0, width=0.3)
sns.stripplot(x="Pclass", y="Age", data=titanic,
              color="k", alpha=0.5, size=3)
plt.title("Age distribution by class", fontsize=12)
plt.tight_layout();

In [ ]:
plt.figure(figsize=(6,6))

sns.violinplot(x="Age", y="Pclass", data=titanic,
               split=True, hue="Sex", scale="count", orient="h")
plt.title("Age distribution by class", fontsize=12)
plt.tight_layout();

In [ ]:
with plt.style.context("seaborn-ticks"):
    plt.figure(figsize=(6,6))

    sns.violinplot(x="Age", y="Pclass", data=titanic,
                   split=True, hue="Sex", scale="count", orient="h",
                   palette={"male": "lightsteelblue", "female": "firebrick"})
    plt.title("Age distribution by class", fontsize=12)

    sns.despine(left=True)

    plt.ylabel("class")

    plt.tight_layout();

For comparison:

In [ ]:
plt.figure(figsize=(15, 5))

for pclass in [1, 2, 3]:
    plt.subplot(1, 3, pclass)
    group_counts.loc[pclass].plot(ax=plt.gca())
    plt.ylim(0, 0.35)
    plt.title("Age distribution for Class %i" % pclass, fontsize=12)
plt.tight_layout()